Library Yang Digunakan

In [1]:
%pip install dask
%pip install openpyxl xlrd

In [15]:
%pip install dask pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
import dask.dataframe as dd

Load Data

In [3]:
df_dask = dd.read_csv("dataTrans.csv")

In [4]:
df_dask.head()

,BULAN,TANGGAL,ART_CODE,ART_DESC,BRAND,TILLCODE,SUBCLASS,TOTAL_QTY,NET_SALES,TOTAL_STRUK,TOTAL_MEMBER,TRANS_NO,POS_NO,CASHIER
0,Mei,01/05/2023,522894,HAPPYDENT WHITE BAG 84G_1C24P_826986,HAPPYDENT,3252536,FOOD,1.0,12500,1,1,1,21,178411
1,Mei,01/05/2023,161089,YUPI NEON STIX 45GR_1C72P,YUPI,2591445,FOOD,6.0,31240,5,4,1,4,178403
2,Mei,01/05/2023,141989,MENTOS ROLL GRAPE 37G_1B24P_631511,MENTOS,3264270,FOOD,2.0,8756,2,1,1,4,178403
3,Mei,01/05/2023,140228,FROZZ BLUEBERRY MINT 15G_1B24P_1C6B,FROZZ,3260272,FOOD,2.0,17200,2,0,1,4,178403
4,Mei,01/05/2023,794294,YUPI LITTLE STARS 45G_1C72P,YUPI,46619,FOOD,2.0,10448,2,1,1,4,178403


In [5]:
# Pilih kolom-kolom yang Anda inginkan
selected_columns = ['TANGGAL', 'ART_CODE', 'ART_DESC', 'TOTAL_QTY', 'NET_SALES', 'TRANS_NO', 'POS_NO']

In [6]:
# Buat DataFrame baru hanya dengan kolom-kolom yang dipilih
selected_df = df_dask[selected_columns]
selected_df.head()

,TANGGAL,ART_CODE,ART_DESC,TOTAL_QTY,NET_SALES,TRANS_NO,POS_NO
0,01/05/2023,522894,HAPPYDENT WHITE BAG 84G_1C24P_826986,1.0,12500,1,21
1,01/05/2023,161089,YUPI NEON STIX 45GR_1C72P,6.0,31240,1,4
2,01/05/2023,141989,MENTOS ROLL GRAPE 37G_1B24P_631511,2.0,8756,1,4
3,01/05/2023,140228,FROZZ BLUEBERRY MINT 15G_1B24P_1C6B,2.0,17200,1,4
4,01/05/2023,794294,YUPI LITTLE STARS 45G_1C72P,2.0,10448,1,4


In [7]:
# Menggabungkan informasi kasir ke nomor transaksi
selected_df['TRANS_UNIQUE'] = selected_df['TRANS_NO'].astype(str) + '_' + selected_df['POS_NO'].astype(str)


In [8]:
selected_df.head()

,TANGGAL,ART_CODE,ART_DESC,TOTAL_QTY,NET_SALES,TRANS_NO,POS_NO,TRANS_UNIQUE
0,01/05/2023,522894,HAPPYDENT WHITE BAG 84G_1C24P_826986,1.0,12500,1,21,1_21
1,01/05/2023,161089,YUPI NEON STIX 45GR_1C72P,6.0,31240,1,4,1_4
2,01/05/2023,141989,MENTOS ROLL GRAPE 37G_1B24P_631511,2.0,8756,1,4,1_4
3,01/05/2023,140228,FROZZ BLUEBERRY MINT 15G_1B24P_1C6B,2.0,17200,1,4,1_4
4,01/05/2023,794294,YUPI LITTLE STARS 45G_1C72P,2.0,10448,1,4,1_4


In [9]:
# Menghitung jumlah transaksi unik menggunakan kombinasi tanggal, nomor transaksi, dan identitas kasir
transaksi_unik = selected_df.groupby(['TANGGAL', 'TRANS_UNIQUE']).size().reset_index().rename(columns={0: 'count'}).compute()

In [10]:
transaksi_unik

,TANGGAL,TRANS_UNIQUE,count
0,01/05/2023,100_17,23
1,01/05/2023,101_17,11
2,01/05/2023,102_17,14
3,01/05/2023,103_17,18
4,01/05/2023,104_17,22
...,...,...,...
59421,31/08/2023,50079_4,2
59422,31/08/2023,50080_21,14
59423,31/08/2023,50080_4,6
59424,31/08/2023,50081_21,3


In [11]:
# Menggabungkan informasi produk ke DataFrame transaksi_unik
transaksi_unik = dd.merge(transaksi_unik, selected_df[['TANGGAL', 'TRANS_UNIQUE', 'ART_DESC']], on=['TANGGAL', 'TRANS_UNIQUE'], how='left')


In [12]:
transaksi_unik.head()

,TANGGAL,TRANS_UNIQUE,count,ART_DESC
0,01/05/2023,100_17,23,MILO UHT 110ML_1C36P
1,01/05/2023,100_17,23,CLEVO COKLAT 125ML_1C40P
2,01/05/2023,100_17,23,ULTRA CHOCOLATE 200 ML_1C24P
3,01/05/2023,100_17,23,REAL GOOD SWEET CHEESE 125ML_1C40P
4,01/05/2023,100_17,23,INDOMILK UHT FULL CREAM 250ML_1C24P


In [14]:
# Mengambil 3000 baris pertama
transaksi_unik_subset = transaksi_unik.head(25000)

In [15]:
# Mengelompokkan berdasarkan kombinasi TANGGAL dan TRANS_UNIQUE
transactions_grouped = transaksi_unik_subset.groupby(['TANGGAL', 'TRANS_UNIQUE'])['ART_DESC'].apply(list).reset_index(name='transaction_list')

In [16]:
# Konversi list menjadi DataFrame
transactions_df = pd.DataFrame(transactions_grouped)
transactions_df.head()

,TANGGAL,TRANS_UNIQUE,transaction_list
0,01/05/2023,100_17,"[MILO UHT 110ML_1C36P, CLEVO COKLAT 125ML_1C40..."
1,01/05/2023,101_17,"[PRONAS CORNED BEEF 340GR_1C24P, PRONAS CORNED..."
2,01/05/2023,102_17,"[RICHEESE MIE RAMEN KEJU LV 0 65 GR_1C40P, MIG..."
3,01/05/2023,103_17,"[WONHAE CHEESE RAMYUN 120G_1C40P, GAGA 100 EXT..."
4,01/05/2023,104_17,"[MUJIGAE JAJANGMYEON 265G_1C40P, GAGA A1 MIE T..."


In [17]:
transactions_df.shape

(438, 3)

In [18]:
# Menggunakan TransactionEncoder untuk mengonversi ke format FP-Growth
te = TransactionEncoder()
te_ary = te.fit(transactions_df['transaction_list']).transform(transactions_grouped['transaction_list'])

# Mengonversi matriks boolean ke dalam matriks numerik (0 dan 1)
te_ary_numeric = te_ary.astype(int)



In [19]:
# Membuat DataFrame yang sesuai dengan format FP-Growth
df_fp = pd.DataFrame(te_ary_numeric, columns=te.columns_, index=transactions_grouped['TRANS_UNIQUE'])
df_fp.head()

,234 SUPER PREMIUM MAGNUM FILTER 12 _1B10P,A&W CAN 250 ML_1C24P,ABC ALK ALKALINE MILP LR 3AAA2S,ABC ALK ALKALINE MILP LR 3AAA4S,ABC ALK ALKALINE MILP LR 6AA 2S,ABC EXO CHOCOMALT COFFEE 200ML_1C24P,ABC EXO MILK COFFEE 200ML_1C24P,ABC KECAP ASIN 135ML_1C48P,ABC KECAP MANIS 133ML_1C48P,ABC KECAP MANIS 275ML_1C24P,...,ZWITSAL BABY BATH NATURAL MLKNHNYPCH450ML_1C12P,ZWITSAL BABY HAIR LOTION NATURAL AVKS100ML _1C24P,ZWITSAL BABY SHAMPOO NAT AVKS 100ML_1C36P,ZWITSAL BABY WIPES ALOE BND 40S_1C12P_68586307,ZWITSAL BABYBATH ANTIBACTERIAL 450ML_68456892_1C12,ZWITSAL BODYMIST 100ML 68285729 _ 1C12P,ZWITSAL KIDS SHP BLUE CLNNFR B180 _1C24P_67883794,ZWITSAL KIDS SHP GRN NATNNRSH B180_1C24P_67883792,ZWITSAL MINYAK TELON 100ML _1C24P,ZWITSAL TWINPACK HAIRBODY 6X_2X450ML_68742014
TRANS_UNIQUE,,,,,,,,,,,,,,,,,,,,,
100_17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101_17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102_17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103_17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
104_17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# Menerapkan algoritma FP-Growth
frequent_itemsets = fpgrowth(df_fp, min_support=0.0002, use_colnames=True)

c:\Users\Calvin Matulessy\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


MemoryError: 

In [73]:
print(frequent_itemsets)

Empty DataFrame
Columns: [support, itemsets]
Index: []


In [62]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

# Contoh data transaksi
data = {'TANGGAL': ['2023-01-01', '2023-01-01', '2023-01-02', '2023-01-02'],
        'TRANS_UNIQUE': ['1_A', '2_B', '3_A', '4_B'],
        'ART_DESC': ['Product_A', 'Product_B', 'Product_A', 'Product_C']}

df = pd.DataFrame(data)

# Mengelompokkan berdasarkan TRANS_UNIQUE
transactions_grouped = df.groupby('TRANS_UNIQUE')['ART_DESC'].apply(list).reset_index(name='transaction_list')

# Menggunakan TransactionEncoder untuk mengonversi ke format FP-Growth
te = TransactionEncoder()
te_ary = te.fit(transactions_grouped['transaction_list']).transform(transactions_grouped['transaction_list'])

# Mengonversi matriks boolean ke dalam matriks numerik (0 dan 1)
te_ary_numeric = te_ary.astype(int)

# Membuat DataFrame yang sesuai dengan format FP-Growth
df_fp = pd.DataFrame(te_ary_numeric, columns=te.columns_, index=transactions_grouped['TRANS_UNIQUE'])

print(df_fp)


              Product_A  Product_B  Product_C
TRANS_UNIQUE                                 
1_A                   1          0          0
2_B                   0          1          0
3_A                   1          0          0
4_B                   0          0          1
